<a href="https://colab.research.google.com/github/WengGeyu/NMA-DL/blob/main/load_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install -q condacolab
import condacolab
condacolab.install_miniconda() # expect a kernel restart

✨🍰✨ Everything looks OK!


In [ ]:
# try to run the bare minimum to get a new conda env working
conda_path = ''
try:
    conda_path = !which conda
finally:
    print('')

if (len(conda_path) == 0):
    print('installing miniconda')
    !wget https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh && bash Miniconda3-4.5.4-Linux-x86_64.sh -bfp /usr/local
    !conda update conda -y -q
    !source /usr/local/etc/profile.d/conda.sh
    !conda init
    !conda install -n root _license -y -q
else:
    print('found miniconda')

conda_envs = !conda env list
res = [i for i in conda_envs if 'test36' in i]
if (len(res) == 0):
    print('not found test36 env', len(res))
    !conda create -y -q --name test36 python=3.6 libarchive cartopy
else:
    print('found test36 env', len(res))

In [ ]:
%%bash
source activate test36
conda env list
conda list
python
import sys
import cartopy
#import libarchive
# maybe only need this the first time we run this notebook
sys.path.append('/usr/local/lib/python3.6/site-packages')

print("Python version")
print(sys.version)

In [ ]:
import os
os.chdir('/content/drive/MyDrive/ColabNotebooks/Aneurysm_Detection-main/install')
!conda env update -f environment_clean.yml

In [ ]:
# !pip install openneuro-py
# import openneuro as on
# on.download(dataset='ds003949', target_dir='/content/drive/MyDrive/ColabNotebooks')
os.chdir('/content/drive/MyDrive/ColabNotebooks/Aneurysm_Detection-main/dataset')
!openneuro-py download --dataset=ds003949 \
                      --include=sub-003 \
                      --include=sub-002 \
                      --include=derivatives/registrations/vesselMNI_2_angioTOF/sub-003 \
                      --include=derivatives/registrations/vesselMNI_2_angioTOF/sub-001 \
                      --include=derivatives/registrations/vesselMNI_2_angioTOF/sub-002 \
# !pip install datalad
# !datalad install https://github.com/OpenNeuroDatasets/ds003949.git


👋 Hello! This is openneuro-py 2023.1.0. Great to see you! 🤗

   👉 Please report problems 🤯 and bugs 🪲 at
      https://github.com/hoechenberger/openneuro-py/issues

🌍 Preparing to download ds003949 …
📁 Traversing directories for ds003949 : 15 entities [02:20,  6.75s/ entities]

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
import os
os.chdir('/content/drive/MyDrive/ColabNotebooks/Aneurysm_Detection-main/install')
!pip install -r env.txt
!pip install SimpleITK
!pip install connected-components-3d
!pip install antspyx
!pip install opencv-python

In [13]:
os.chdir('/content/drive/MyDrive/ColabNotebooks/Aneurysm_Detection-main/dataset_creation')
%run create_dataset_patches_neg_and_pos.py --config config_creation_ds_patches.json


Computing intensity thresholds...


AssertionError: ignored

In [ ]:
!git clone https://github.com/MIC-DKFZ/nnUNet.git
%cd nnUNet
!pip install -e .
# nnUNetv2_plan_and_preprocess -d DATASET_ID --verify_dataset_integrity

In [1]:
# !pip3 install http://download.pytorch.org/whl/cu80/torch-0.4.0-cp36-cp36m-linux_x86_64.whl
# !pip3 install torchvision
# !pip3 install numpy==1.14.1

# Import all the necessary packages
import numpy as np
import nibabel as nib
# import itk
# import itkwidgets
from ipywidgets import interact, interactive, IntSlider, ToggleButtons
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('darkgrid')

from collections import defaultdict
import time
import os

from tqdm import tqdm

import torch
from torchvision import transforms
from torch.utils.data import DataLoader
from torch import autograd, optim
import torch.backends.cudnn as cudnn
import torch.nn as nn

import torchvision.utils as vutils

cudnn.benchmark = True
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
input_path = "/content/drive/MyDrive/ColabNotebooks/Aneurysm_Detection-main/dataset/derivatives/N4_bias_field_corrected/sub-000/ses-20110101/anat"
input_filename = os.path.join(input_path,'sub-000_ses-20110101_desc-angio_N4bfc_mask.nii.gz')
image_obj = nib.load(input_filename)
print(f'Type of the image {type(image_obj)}')

Type of the image <class 'nibabel.nifti1.Nifti1Image'>


In [ ]:
# Extract data as numpy ndarray
image_data = image_obj.get_fdata()
# Get the image shape and print it out
height, width, depth = image_data.shape
print(f"The image object has the following dimensions: height: {height}, width:{width}, depth:{depth}")
# Select random layer number
maxval = depth
i = np.random.randint(0, maxval)
# Define a channel to look at
channel = 0
print(f"Plotting Layer {i} Channel {channel} of Image")
plt.imshow(image_data[:, :, i], cmap='gray')
plt.axis('off');